In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
'exec(%matplotlib inline)'
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(11) # It's my lucky number
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
import itertools

import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from keras.models import load_model

In [ ]:
folder_benign_train = '/content/drive/My Drive/Colab Notebooks/data/train/benign'
folder_malignant_train = '/content/drive/My Drive/Colab Notebooks/data/train/malignant'

folder_benign_test = '/content/drive/My Drive/Colab Notebooks/data/test/benign'
folder_malignant_test = '/content/drive/My Drive/Colab Notebooks/data/test/malignant'

read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
# Load in training pictures 
ims_benign = [read(os.path.join(folder_benign_train, filename)) for filename in os.listdir(folder_benign_train)]
X_benign = np.array(ims_benign, dtype='uint8')
print("load benign training pictures complete.")
ims_malignant = [read(os.path.join(folder_malignant_train, filename)) for filename in os.listdir(folder_malignant_train)]
X_malignant = np.array(ims_malignant, dtype='uint8')
print("load malignant training pictures complete.")

# Load in testing pictures
ims_benign = [read(os.path.join(folder_benign_test, filename)) for filename in os.listdir(folder_benign_test)]
X_benign_test = np.array(ims_benign, dtype='uint8')
print("load benign testing pictures complete.")
ims_malignant = [read(os.path.join(folder_malignant_test, filename)) for filename in os.listdir(folder_malignant_test)]
X_malignant_test = np.array(ims_malignant, dtype='uint8')
print("load malignant testing pictures complete.")

# Create labels
y_benign = np.zeros(X_benign.shape[0])
y_malignant = np.ones(X_malignant.shape[0])
y_benign_test = np.zeros(X_benign_test.shape[0])
y_malignant_test = np.ones(X_malignant_test.shape[0])
print("create labels complete.")

# Merge data 
X_train = np.concatenate((X_benign, X_malignant), axis = 0)
y_train = np.concatenate((y_benign, y_malignant), axis = 0)
X_test = np.concatenate((X_benign_test, X_malignant_test), axis = 0)
y_test = np.concatenate((y_benign_test, y_malignant_test), axis = 0)
print("merge data complete.")

# Shuffle data
s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
y_train = y_train[s]

s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
y_test = y_test[s]

y_train = to_categorical(y_train, num_classes= 2)
y_test = to_categorical(y_test, num_classes= 2)

model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),padding = 'Same',input_shape=(224,224,3), activation= 'relu', kernel_initializer='glorot_uniform'))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3),padding = 'Same', activation ='relu', kernel_initializer = 'glorot_uniform'))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(132, activation='relu', kernel_initializer='normal'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs= 50, batch_size= 64, verbose=1)
model.save('/content/drive/My Drive/skinCaner.h5')

load benign training pictures complete.
load malignant training pictures complete.
load benign testing pictures complete.
load malignant testing pictures complete.
create labels complete.
merge data complete.
Epoch 1/50
2637/2637 [==============================] - 16s 6ms/step - loss: 50.2064 - accuracy: 0.5021
Epoch 2/50
2637/2637 [==============================] - 7s 3ms/step - loss: 0.7042 - accuracy: 0.5119
Epoch 3/50
2637/2637 [==============================] - 7s 3ms/step - loss: 0.6979 - accuracy: 0.5216
Epoch 4/50
2637/2637 [==============================] - 7s 3ms/step - loss: 0.6909 - accuracy: 0.5305
Epoch 5/50
2637/2637 [==============================] - 7s 3ms/step - loss: 0.6830 - accuracy: 0.5468
Epoch 6/50
2637/2637 [==============================] - 7s 3ms/step - loss: 0.6556 - accuracy: 0.5846
Epoch 7/50
2637/2637 [==============================] - 7s 3ms/step - loss: 0.5936 - accuracy: 0.6363
Epoch 8/50
2637/2637 [==============================] - 7s 3ms/step - loss:

In [ ]:
y_pred = model.predict_classes(X_test)
res = 0
for i in range(0, len(y_pred)):
  print(y_pred[i]==y_test[i][1])
  if (y_pred[i]==y_test[i][1]):
    res+=1
print(res/len(y_pred))

False
True
True
True
True
True
True
True
True
False
True
False
True
True
True
False
True
True
True
True
False
True
False
True
True
True
True
True
True
True
True
True
False
False
False
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
False
True
True
True
False
True
True
False
True
True
True
False
True
True
True
True
True
False
True
True
False
True
False
True
False
True
True
False
True
True
True
False
False
True
True
True
True
True
True
True
False
False
True
False
False
True
True
False
True
False
True
True
False
True
True
True
False
True
False
True
True
True
True
True
True
False
True
False
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
False
True
True
False
True
True
True
False
True
False
False
True
True
True
True
True
True
True
False
True
True
True
True
False
False
True
True
True
True
True
True
True
True
True
False
True
False
True
False
True
True
True
False
True